# Практика
Используемые библиотеки

In [3]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [25]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [36]:
# Ваше решение
rdf=df
# rdf = rdf.loc[:,'Норматив']
rdf['col1']=rdf['Норматив'].str.split(' ').str.get(0) + ' ' + rdf['Норматив'].str.split(' ').str.get(1)
rdf['col2']=rdf['Норматив'].str.split(' ').str.get(2)
rdf['col3']=rdf['col2'].str.split('-').str.get(0)
rdf['col3'] = rdf['col3'].str.replace(',','.')
rdf['col3'] = pd.to_numeric(rdf['col3'], errors='coerce')
rdf['col4']=rdf['col2'].str.split('-').str.get(1)
rdf['col4'] = rdf['col4'].str.replace(',','.')
rdf['col4'] = pd.to_numeric(rdf['col4'], errors='coerce')
rdf['Результат анализа'] = pd.to_numeric(rdf['Результат анализа'], errors='coerce')

def function(row):
    if pd.isnull(row['Результат анализа']):
        return 'нет данных'
    else:
        # return 'есть данные'
        if 'не более' in row['col1']:
            if row['Результат анализа'] <= row['col3']:
                return 'в норме'
            else:
                return 'не в норме'
        elif 'в пределах' in row['col1']:
            if row['col3'] <= row['Результат анализа'] <= row['col4']:
                return 'в норме'
            else:
                return 'не в норме'
        else:
            return 'нет данных'

rdf['Ответ'] = rdf.apply(function, axis=1)
rdf

,Показатель,Единица измерений,Результат анализа,Норматив,col1,col2,col3,col4,Ответ
0,pH,единицы pH,8.400,в пределах 6-9,в пределах,6-9,6.0,9.0,в норме
1,Запах,баллы,1.000,не более 2-3,не более,2-3,2.0,3.0,в норме
2,Цветность,градусы,NaN,не более 30,не более,30,30.0,NaN,нет данных
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7-10,7.0,10.0,в норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",не более,"1,5",1.5,NaN,в норме
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,"3,3",3.3,NaN,в норме
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45,45.0,NaN,в норме
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",не более,"3,5",3.5,NaN,в норме
8,Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350,350.0,NaN,в норме
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,не более,500,500.0,NaN,в норме


В итоговом DataFrame столбец "Показатель" сделаем индексным и  уберем лишние столбцы.

In [44]:
rdf.set_index('Показатель')
rdf.loc[:, ['Единица измерений','Результат анализа','Норматив','Ответ']]

,Единица измерений,Результат анализа,Норматив,Ответ
0,единицы pH,8.400,в пределах 6-9,в норме
1,баллы,1.000,не более 2-3,в норме
2,градусы,NaN,не более 30,нет данных
3,мг-эквл/дм3,9.200,в пределах 7-10,в норме
4,мг/дм3,0.420,"не более 1,5",в норме
5,мг/дм3,0.017,"не более 3,3",в норме
6,мг/дм3,24.000,не более 45,в норме
7,мг/дм3,0.360,"не более 3,5",в норме
8,мг/дм3,200.000,не более 350,в норме
9,мг/дм3,189.500,не более 500,в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [110]:
# Ваше решение
import random
test_box = ['Apple','Apple','Apple','Apple','Orange','Orange','Orange','Orange','Orange']
n=1000000
i=0
k = 0
# test = []
while i < n:
    i +=1
    test = random.sample(test_box, 3)
    if test == ['Orange','Orange','Orange']:
        k+=1
    # print(test)
print('P(3 orange) = ', k/n)

P(3 orange) =  0.119355


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [7]:
# Ваше решение


import random
test_box = ['Standart','Standart','Standart','Standart','Standart','Standart','Standart','No_standart','No_standart','No_standart']
n=1000000
i=0
k = 0
# test = []
while i < n:
    i +=1
    test = random.sample(test_box, 2)
    if test == ['No_standart','Standart']:
        k+=1
    # print(test)
print('P(No_standart, Standart) = ', k/n)

P(No_standart, Standart) =  0.234007


In [ ]:
# Например,
f = ["Апельсин" for x in range(5)] + ["Яблоко" for y in range(4)]
def shuffle (f=f):
    np.random.shuffle(f)
    return f[:3]
fruits = pd.DataFrame(list(map(lambda x: shuffle(f), range(N))),
                      columns=["Fruit1", "Fruit2", "Fruit3"])
